# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# Importing the dataset
orders = pd.read_csv("Orders.zip")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# Aggregate amount_sment for unique customers. 
# Aggregate by cutomer id then sum total amount 
customer_total = orders.groupby(['CustomerID','Country']).agg({'amount_spent':sum})
customer_total

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [4]:
# Select customers whose aggregated amount_spent is in a given quantile range?
q75 = customer_total['amount_spent'].quantile(q=0.75)
q95 = customer_total['amount_spent'].quantile(q=0.95)
print(q75)
print(q95)
customer_total['vip_customer'] = customer_total['amount_spent'] > q95
customer_total

1661.195
5774.277999999996
5154.147599999999


,,amount_spent,vip_customer
CustomerID,Country,,
12346,United Kingdom,77183.60,True
12347,Iceland,4310.00,False
12348,Finland,1797.24,False
12349,Italy,1757.55,False
12350,Norway,334.40,False
...,...,...,...
18280,United Kingdom,180.60,False
18281,United Kingdom,80.82,False
18282,United Kingdom,178.05,False


In [8]:
customer_total['preferred_customer']=(customer_total['amount_spent'].between(q75,q95))

In [10]:
customer_total

,,amount_spent,vip_customer,preferred_customer
CustomerID,Country,,,
12346,United Kingdom,77183.60,True,False
12347,Iceland,4310.00,False,True
12348,Finland,1797.24,False,True
12349,Italy,1757.55,False,True
12350,Norway,334.40,False,False
...,...,...,...,...
18280,United Kingdom,180.60,False,False
18281,United Kingdom,80.82,False,False
18282,United Kingdom,178.05,False,False


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [11]:
# your code here
customer_total = customer_total.reset_index()
customer_total

,CustomerID,Country,amount_spent,vip_customer,preferred_customer
0,12346,United Kingdom,77183.60,True,False
1,12347,Iceland,4310.00,False,True
2,12348,Finland,1797.24,False,True
3,12349,Italy,1757.55,False,True
4,12350,Norway,334.40,False,False
...,...,...,...,...,...
4342,18280,United Kingdom,180.60,False,False
4343,18281,United Kingdom,80.82,False,False
4344,18282,United Kingdom,178.05,False,False
4345,18283,United Kingdom,2094.88,False,True


In [12]:
customer_total = customer_total.replace(True, 1)
customer_total = customer_total.replace(False, 0)
customer_total['vip_customer'].value_counts()

0    4129
1     218
Name: vip_customer, dtype: int64

In [13]:
vip_customer_count=customer_total.groupby(['Country']).agg({'vip_customer':np.sum})
vip_customer_count[vip_customer_count['vip_customer']==vip_customer_count['vip_customer'].max()]
#UK has most VIP customers

,vip_customer
Country,
United Kingdom,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [14]:
customer_total['combined']= customer_total['vip_customer'] +customer_total['preferred_customer']
customer_total

,CustomerID,Country,amount_spent,vip_customer,preferred_customer,combined
0,12346,United Kingdom,77183.60,1,0,1
1,12347,Iceland,4310.00,0,1,1
2,12348,Finland,1797.24,0,1,1
3,12349,Italy,1757.55,0,1,1
4,12350,Norway,334.40,0,0,0
...,...,...,...,...,...,...
4342,18280,United Kingdom,180.60,0,0,0
4343,18281,United Kingdom,80.82,0,0,0
4344,18282,United Kingdom,178.05,0,0,0
4345,18283,United Kingdom,2094.88,0,1,1


In [15]:
vip_preferred=customer_total.groupby(['Country']).agg({'combined':np.sum})
vip_preferred[vip_preferred['combined']==vip_preferred['combined'].max()]
#UK has most combined customers

,combined
Country,
United Kingdom,934
